word2vec tutorial
====

1. 간단한 이론적 소개
2. word2vec 설치
3. 영어 코퍼스로 맛보기
4. gensim으로 요리하기
5. KoNLPy 및 mecab-ko 설치
6. 한국어 코퍼스로 word2vec 생성
7. 의미 코퍼스로 학습 집합 생성
8. 단어 의미 중의성 해결 - Part 1 - 전통적인 방법
9. 단어 의미 중의성 해결 - Part 2 - word2vec 활용

1. 간단한 이론적 소개
----

### Efficient Estimation of Word Representations in Vector Space
* 논문: http://arxiv.org/pdf/1301.3781.pdf
* 발표 자료: https://drive.google.com/file/d/0B7XkCwpI5KDYRWRnd1RzWXQ2TWc/edit?usp=sharing

### Backgrounds(Prerequisite)
* Perceptron: http://en.wikipedia.org/wiki/Perceptron
* Artificial Neural Network(ANN): http://en.wikipedia.org/wiki/Artificial_neural_network
  - Back-propagation: http://en.wikipedia.org/wiki/Backpropagation
* Deep Neural Network(DNN): http://en.wikipedia.org/wiki/Deep_learning#Deep_neural_networks
  - feed-foward NN: http://en.wikipedia.org/wiki/Feedforward_neural_network
  - Recurrent NN(RNN): http://en.wikipedia.org/wiki/Recurrent_neural_network
  - feed-forward vs recurrent: http://stats.stackexchange.com/questions/2213/feed-forward-and-recurrent-neural-networks

### Text(word) Representation
* Local representation
  - n-grams
  - bag-of-words
  - 1-of-N coding: http://en.wikipedia.org/wiki/Constant-weight_code#1_of_N_codes
* Continuous representation
  - Latent Semantic Index(LSA)
  - Latent Dirichlet Allocation(LDA)
  - Distributed representation

### Training complexity(O)
* O = E x T x Q
  - E: number of the training epochs (iteration, 3 ~ 50)
  - T: number of the words in the training set (~ 1 billion)
  - Q: defined further for each model architecture

### Previous works
#### feed-forward Neural Net Language Model(NNLM)
![](img/feedforwardNNLM.png)
* predicts current word by previous n words
* the word vectors are in matrix U
* trained by stochastic gradient descent and back-propagation
* complexity
  - Q = N x D + N x D x H + H x V
    * N: n previous words (n-gram,  ~ 10)
    * D: size of projection layer (dimension, 500 ~ 1000)
    * H: size of hidden layer
    * V: size of vocabulary
  - dominant term: N x D x H => very complex
  - for efficient learning
    * hierarchical softmax: V => log2(V)
    * ___negative sampling___
      - word2vec explained: http://arxiv.org/abs/1402.3722

#### Recurrent Neural Net Language Model(RNNLM)
![](img/rnnlm.png)
* does not have projection layer; only input, hidden and output layer
* recurrent matrix that connects hidden layer to itself, using time-delayed connections
* complexity
  - Q = H x H + H x V
  - dominant term: H x H
* http://research.microsoft.com/pubs/189726/rvecs.pdf

### New Log-linear Models
#### Continuous Bag-of-Words(CBOW) model
![](img/cbow.png)
* predicts the current word given the context
* ___the non-linear hidden layer is removed___ and the projection layer is shared for all words
* the order of words in the history does not influence the projection
* complexity
  - Q = N x D + D x log2(V)

#### Continuous skip-gram model
![](img/skip-gram.png)
* predicts the surrounding words given the current word
* tries to maximize classification of a word based on another word in the same sentence
* complexity
  - Q = C x (D + D x log2(V))
    * C: maximum distance of the words (window size, 10)

### Experiments
#### Measure
* Linguistic regularities
  - "What is the word that is similar to 'small' in in the same sense as 'biggest' is similar to 'big'?"
    * compute vector X = vector('biggest') - vector('big') + vector('small')
    * closest vector from X is vector('smallest')
* semantic and syntactic regularities
![](img/linguistic-regularities.png)
* calcuates the accuracy

#### Evaluation set
* Semantic-Syntactic Word Relationship test set (20K questions)
![](img/semantic-syntactic-types.png)
* The Microsoft Research Sentence Completion Challenge
  - http://research.microsoft.com/apps/pubs/default.aspx?id=157031

#### Results
* 20K questions set
![](img/results.png)
* Microsoft set
![](img/microsoft.png)

2. word2vec 설치
----

### 다운로드

```
$ svn checkout http://word2vec.googlecode.com/svn/trunk/ word2vec
```

### 빌드
#### Linux

```
$ cd word2vec
$ make
```

#### Mac
* makefile 내용 수정

```makefile
# CFLAGS = -lm -pthread -O3 -march=native -Wall -funroll-loops -Wno-unused-result
CFLAGS = -lm -pthread -O3 -Wall -funroll-loops    # -march=native, -Wno-unused-result 제거
```

* 소스코드 수정

```
$ grep malloc.h *.c
compute-accuracy.c:#include <malloc.h>
distance.c:#include <malloc.h>
word-analogy.c:#include <malloc.h>
```
malloc.h를 include하는 코드를 모두 지워버립니다. 혹시 아까우시다면 주석 처리를 해도 괜찮아요. ^^;


* 빌드

```
$ make
```

3. 영어 코퍼스로 맛보기
----

* 샘플 코퍼스 다운로드
```
$ wget http://mattmahoney.net/dc/text8.zip
$ unzip text8.zip
```

* word vector 생성

```
$ ./word2vec -train text8 -output text8.w2v -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-4 -threads 4 -binary 1 -iter 15
Starting training using file text8
Vocab size: 71291
Words in train file: 16718843
Alpha: 0.000005  Progress: 100.02%  Words/thread/sec: 92.08k

$ ls -l text8.w2v
-rw-r--r--  1 jamie  staff  57704809 11  6 01:12 text8.w2v
```
-threads argument는 CPU 코어의 갯수에 맞게 적절히 넣으면 됩니다.


* word analogy 해보기

A B C 세 단어를 (소문자로) 넣으면 vector(A) - vector(B) + vector(C)를 계산하여 벡터간 거리로 정렬하여 가장 가까운 단어부터 보여줍니다. (우왕ㅋ굳ㅋ)

```
$ ./word-analogy text8.w2v
Enter three words (EXIT to break): athens greece baghdad

Word: athens  Position in vocabulary: 3068

Word: greece  Position in vocabulary: 1248

Word: baghdad  Position in vocabulary: 7173

                                              Word              Distance
------------------------------------------------------------------------
                                              iran		0.490723
                                             syria		0.488516
                                             egypt		0.458451
                                            turkey		0.450124
                                             basra		0.448245
                                              iraq		0.444451
                                        sassanians		0.438153
                                             mosul		0.431619
                                           lebanon		0.428784
                                         kurdistan		0.426602
                                              sadr		0.423816
                                            persia		0.423356
                                             konya		0.421188
                                           abbasid		0.415041
                                            khitan		0.410358
                                           bukhara		0.410169
                                            kirkuk		0.405520
                                           fatimid		0.404706
                                           iranian		0.404393
                                           ayyubid		0.404103
                                           mameluk		0.403826
                                             libya		0.400661
                                           almohad		0.400090
                                          pakistan		0.398301
                                          iranians		0.397244
                                              arab		0.395184
                                              zmir		0.394183
                                            zagros		0.394126
                                            mashal		0.393248
                                             balkh		0.393002
                                       afghanistan		0.391483
                                           tunisia		0.389862
                                           tahmasp		0.389630
                                        uzbekistan		0.389564
                                            tigris		0.389527
                                        tajikistan		0.389425
                                           emirate		0.388417
                                        kermanshah		0.388197
                                           khanate		0.385721
                                             hejaz		0.384893
```

* 샘플 word analogy 쿼리

```
$ grep -A1 "^:" questions-words.txt
: capital-common-countries
Athens Greece Baghdad Iraq
--
: capital-world
Abuja Nigeria Accra Ghana
--
: currency
Algeria dinar Angola kwanza
--
: city-in-state
Chicago Illinois Houston Texas
--
: family
boy girl brother sister
--
: gram1-adjective-to-adverb
amazing amazingly apparent apparently
--
: gram2-opposite
acceptable unacceptable aware unaware
--
: gram3-comparative
bad worse big bigger
--
: gram4-superlative
bad worst big biggest
--
: gram5-present-participle
code coding dance dancing
--
: gram6-nationality-adjective
Albania Albanian Argentina Argentinean
--
: gram7-past-tense
dancing danced decreasing decreased
--
: gram8-plural
banana bananas bird birds
--
: gram9-plural-verbs
decrease decreases describe describes
```

4. gensim으로 요리하기
----


### gensim 설치

```
$ pip install gensim
```
끝 (우왕ㅋ굳ㅋ)

* 참고
  - https://radimrehurek.com/gensim/models/word2vec.html
  - http://rare-technologies.com/word2vec-tutorial/

In [ ]:
### word2vec 모델을 만들어 봅시다.

import gensim

sentences = sum([line.split() for line in open('text8')], [])    # text8 코퍼스를 모두 읽어들입니다.
print sentences[:10]

# C 프로그램과 유사하게 학습을 실행합니다.
gensim_model = gensim.models.Word2Vec(sentences, size=200, window=8, min_count=5, workers=4, iter=15)

gensim_model.save('text8.w2v.gensim')    # 학습한 모델을 저장합니다.
gensim_new_model = gensim.models.Word2Vec.load('text8.w2v.gensim')    # 저장한 모델을 읽어들입니다.

# 하지만 오래 살기 위해 이러지 말고 Jeff Dean을 믿고 씁시다.

![](http://static1.businessinsider.com/image/51098904ecad04ad07000027/meet-googles-baddest-engineer-jeff-dean.jpg)

In [ ]:
### 기존에 C 프로그램으로 학습한 모델을 읽어들여 여러가지 연산을 해봅니다.

import gensim

gensim_model = gensim.models.Word2Vec.load_word2vec_format('text8.w2v', binary=True)    # 모델을 읽어들입니다.

# word analogy
print '* woman - man + king =', gensim_model.most_similar(positive=[u'woman', u'king'], negative=[u'man'], topn=3)
print

# 미운 오리 새끼를 찾습니다. ;)
print '* The lone duckling =', gensim_model.doesnt_match(u'breakfast cereal dinner lunch'.split())
print

# 두 단어의 유사도를 계산합니다.
print '* Similarity between woman and man =', gensim_model.similarity(u'woman', u'man')
print

# 단어의 벡터를 출력합니다.
print '* Vector of man:'
print gensim_model[u'man']

* woman - man + king = [(u'queen', 0.5841079354286194), (u'betrothed', 0.5011240243911743), (u'throne', 0.48706796765327454)]

* The lone duckling = cereal

* Similarity between woman and man = 0.599900135036

* Vector of man:
[-0.02120703 -0.05611495 -0.0448945  -0.10382977  0.06347878 -0.03084225
 -0.05465495 -0.06092014  0.0411589  -0.01161192 -0.03177946 -0.00450953
  0.08355674  0.08220534  0.09392931  0.07430327 -0.09978093  0.0255994
  0.06163046 -0.00767307  0.0402733  -0.0438408  -0.09186259 -0.070136
 -0.13232407  0.01343085  0.02881281 -0.01540345  0.10018467  0.02999429
 -0.04141324  0.03193626  0.04727618  0.03440831 -0.05067592 -0.13143836
  0.04143735  0.08112337  0.09542105 -0.01772195  0.07869302  0.03986564
 -0.03947217 -0.04002963  0.12870036 -0.00388008 -0.11321796  0.03314336
  0.08151811  0.12611714  0.10488386  0.026107   -0.0391846   0.07439879
  0.04260764 -0.08983206  0.06683791  0.05873458  0.06682455  0.06476297
  0.14592303  0.0288553   0.02531698 -0.147981

5. KoNLPy 및 mecab-ko 설치
----

### konlpy 패키지 설치

```
$ pip install konlpy
```

### mecab-ko 프로그램 설치

* 아래와 같은 마법의 스크립트는 에러가 나니 한땀한땀 해보겠습니다.

```
$ bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
```

* \$HOME/usr/mecab-ko 아래에 설치한다고 가정하고 아래와 같이 소스코드를 다운로드합니다.

```
$ cd $HOME/usr
$ wget https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
```

* 압축을 풀고 빌드/설치를 합니다.

```
$ tar xfz mecab-0.996-ko-0.9.2.tar.gz
$ cd mecab-0.996-ko-0.9.2
$ ./configure --prefix=$HOME/usr/mecab-ko
$ make && make install
```

### mecab-ko 사전 설치

* 역시 \$HOME/usr/mecab-ko 아래에 설치한다고 가정하고 아래와 같이 사전을 다운로드합니다.

```
$ cd $HOME/usr
$ wget https://bitbucket.org/eunjeon/mecab-ko-dic/downloads/mecab-ko-dic-2.0.1-20150920.tar.gz
```

* 압축을 풀고 빌드/설치를 합니다.

```
$ tar xfz mecab-ko-dic-2.0.1-20150920.tar.gz
$ cd mecab-ko-dic-2.0.1-20150920
$ ./autogen.sh
$ ./configure --prefix=$HOME/usr/mecab-ko --with-mecab-config=$HOME/usr/mecab-ko/bin/mecab-config
$ make && make install
```

### 환경 설정

```
$ export LD_LIBRARY_PATH=$HOME/usr/mecab-ko/lib:$LD_LIBRARY_PATH    # Linux의 경우
$ export DYLD_LIBRARY_PATH=$HOME/usr/mecab-ko/lib:$DYLD_LIBRARY_PATH    # Mac의 경우
```

### 테스트

```
$ cd $HOME/usr/mecab-ko
$ bin/mecab
존나좋군?
존나	MAG,*,F,존나,*,*,*,*
좋	VA,*,T,좋,*,*,*,*
군	EF,*,T,군,*,*,*,*
?	SF,*,*,*,*,*,*,*
EOS
```
![](http://pds23.egloos.com/pds/201112/24/41/d0021441_4ef5e7b6eb457.jpg)

### mecab-ko python 모듈 설치

* 소스 코드를 다운로드 합니다.

```
$ cd $HOME/usr
$ git clone https://bitbucket.org/eunjeon/mecab-python-0.996.git
```

* 설치 및 빌드를 합니다.

```
$ cd mecab-python-0.996
$ python setup.py build
$ sudo python setup.py install    # virtualenv를 사용하신다면 sudo는 내려놓으셔도 됩니다.
```

In [ ]:
### KoNLPy에서 mecab-ko를 사용해 봅시다.

from konlpy.tag import Mecab
from konlpy.utils import pprint
import os

# 사전 경로를 주지 않으면 /usr/local/lib/mecab/dic/mecab-ko-dic을 찾습니다.
mecab = Mecab('%s/usr/mecab-ko/lib/mecab/dic/mecab-ko-dic' % os.environ['HOME'])
pprint(mecab.pos(u'존나좋군?'))    # [(형태소1, 태그1), (형태소2, 태그2), ...)] 튜플의 리스트 형태로 리턴합니다.

[(존나, MAG),
 (좋, VA),
 (군, EF),
 (?, SF)]


6. 한국어 코퍼스로 word2vec 생성
----

### 원시 코퍼스

* 원시 코퍼스의 압축을 푼다.

```
$ cat raw_corpus.zip.0 raw_corpus.zip.1 >> raw_corpus.zip
$ unzip -q raw_corpus.zip
```

* 인코딩을 UTF-16에서 UTF-8로 변환하여 하나의 파일로 합친다. => raw_corpus.xml

```
$ iconv -f UTF-16 -t UTF-8 raw_corpus/*.txt > raw_corpus.xml
```

In [ ]:
### 태그를 제거하고 문장만 남긴다. => raw_corpus.txt

def strip_tag(line, tag):
    """
    라인을 둘러싼 열고 닫는 xml 태그를 제거하는 함수
    @param  line  xml 태그를 포함하는 라인
    @param  tag   태그명
    @return       태그가 제거된 라인
    """
    open_tag = '<%s>' % tag
    close_tag = '</%s>' % tag
    stripped = line
    if stripped.startswith(open_tag):
        stripped = stripped[len(open_tag):]
    if stripped.endswith(close_tag):
        stripped = stripped[:-len(close_tag)]
    return stripped.strip()

with open('raw_corpus.txt', 'w') as fout:
    for line in open('raw_corpus.xml'):    # 앞서 xml로 저장한 원시 코퍼스를 열어서
        line = line.strip()
        if not line:
            continue
        if line.startswith('<head>') or line.startswith('<p>'):    # <head>, <p> 태그로 시작하는 문장만
            line = strip_tag(line, 'head')
            line = strip_tag(line, 'p')
            if line:
                print >> fout, line    # 한 줄에 한 문장씩 출력

!head -n5 raw_corpus.txt

"정치 신뢰회복 최선 다하겠다" / 박태준 민정대표 일문일답
"당내 융화-결속엔 별 문제 없어 / 민주화 촉진-경제난 극복에 맞춰 당운영"
민정당의 신임 박태준대표위원은 "기왕에 정치일선에 나선이상 신명을 바쳐 저하된 정치의 신뢰를 회복하는 데 최선을 다하겠다"고 첫 소감을 말했다.
5일 오후 노태우대통령으로부터 대표위원 임명통보를 받은 뒤 오후 4 시쯤 민정당사에 도착, 곧바로 가진 기자회견에서 그는 밝은 표정으로 질문에 응했으나 정계개편등 중요 현안에 대해서는 "앞으로 공부를 더해 답하겠다"는 말로 답변을 대신했다.
- 어려운 시기에 대표위원을 맡았는데….


In [ ]:
### KoNLPy를 이용해 품사 태깅을 수행한다. => raw_corpus.tag

from konlpy.tag import Mecab
from konlpy.utils import pprint
import os

mecab = Mecab('%s/usr/mecab-ko/lib/mecab/dic/mecab-ko-dic' % os.environ['HOME'])

with open('raw_corpus.tag', 'w') as fout:
    for line in open('raw_corpus.txt'):    # 앞서 txt로 저장한 원시 코퍼스를 열어서
        for morph, tag in mecab.pos(unicode(line, 'UTF-8')):    # 형태소 분석을 수행한 다음
            print >> fout, '%s/%s' % (morph.encode('UTF-8'), tag.encode('UTF-8')),    # '정치/NNG'의 형태로
        print >> fout    # 한 줄에 한 문장씩 출력

!head -n5 raw_corpus.tag

"/SY 정치/NNG 신뢰/NNG 회복/NNG 최선/NNG 다/MAG 하/VV 겠/EP 다/EC "/SY //SC 박태준/NNP 민정/NNG 대표/NNG 일문일답/NNG
"/SY 당내/NNG 융화/NNG -/SY 결속/NNG 엔/JKB+JX 별/MM 문제/NNG 없/VA 어/EF //SC 민주/NNG 화/XSN 촉진/NNG -/SY 경제난/NNG 극복/NNG 에/JKB 맞춰/VV+EC 당/NNG 운영/NNG "/SY
민/NNP 정당/NNG 의/JKG 신임/NNG 박태준/NNP 대표/NNG 위원/NNG 은/JX "/SY 기왕/NNG 에/JKB 정치/NNG 일선/NNG 에/JKB 나선/VV+ETM 이상/NNG 신명/NNG 을/JKO 바쳐/VV+EC 저하/NNG 된/XSV+ETM 정치/NNG 의/JKG 신뢰/NNG 를/JKO 회복/NNG 하/XSV 는/ETM 데/NNB 최선/NNG 을/JKO 다/MAG 하/VV 겠/EP 다/EC "/SY 고/JKQ 첫/MM 소감/NNG 을/JKO 말/NNG 했/XSV+EP 다/EF ./SF
5/SN 일/NNBC 오후/NNG 노태우/NNP 대통령/NNG 으로부터/JKB 대표/NNG 위원/NNG 임명/NNG 통보/NNG 를/JKO 받/VV 은/ETM 뒤/NNG 오후/NNG 4/SN 시/NNBC 쯤/XSN 민정/NNG 당사/NNG 에/JKB 도착/NNG ,/SC 곧바로/MAG 가진/VV+ETM 기자/NNG 회견/NNG 에서/JKB 그/NP 는/JX 밝/VA 은/ETM 표정/NNG 으로/JKB 질문/NNG 에/JKB 응했/VV+EP 으나/EC 정계/NNG 개편/NNG 등/NNB 중요/NNG 현안/NNG 에/JKB 대해서/VV+EC 는/JX "/SY 앞/NNG 으로/JKB 공부/NNG 를/JKO 더해/VV+EC 답/NNG 하/XSV 겠/EP 다/EC "/SY 는/JX 말/NNG 로/JKB 답변/NNG 을/JKO 대신/NNG 했/XSV+EP 다/EF ./SF
-/SY 어려운/VA+ETM 시기/NNG 에/JKB 대표/NNG 위원/NNG 을/JK

* 앞서 생성한 자동 태깅한 코퍼스를 이용하여 '단어/태그'의 벡터를 생성한다. => raw_corpus.w2v

```
$ ./word2vec -train raw_corpus.tag -output raw_corpus.w2v -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-4 -threads 4 -binary 1 -iter 15
```

7. 의미 코퍼스로 학습 집합 생성
----

### 의미 코퍼스

* 의미 코퍼스의 압축을 푼다.

```
$ cat sense_corpus.zip.0 sense_corpus.zip.1 >> sense_corpus.zip
$ unzip -q sense_corpus.zip
```

* 인코딩을 UTF-16에서 UTF-8로 변환하여 하나의 파일로 합친다. => sense_corpus.xml

```
$ iconv -f UTF-16 -t UTF-8 sense_corpus/*.txt > sense_corpus.xml
```

In [ ]:
### 특정 단어를 포함하는 문장을 추출하여 단어의 의미 번호와 함께 출력한다. => sense_corpus.tag

TARGET = '연기__'    # 추출을 원하는 단어. 예: 배, 경기, 연기

import re

LINE_ID_PTN = re.compile('[0-9A-Z_]{4}\\d{4}-\\d{7,8}')    # 형태소 분석 된 어절 한 줄의 맨 앞에 ID 패턴

with open('sense_corpus.tag', 'w') as fout:
    sent = []
    for line in open('sense_corpus.xml'):    # 앞서 xml로 저장한 의미 코퍼스를 열어서
        line = line.strip()
        if not line:
            continue
        if LINE_ID_PTN.match(line):    # 다른 라인은 버리고 어절만을 취하여
            _, morphs = line.rsplit('\t', 1)
            sent.extend(morphs.split(' + '))
        else:
            # 한 문장에서 원하는 단어의 의미 번호를 모두 추출하여
            sense_nums = set([morph[len(TARGET):morph.rindex('/')] for morph in sent if morph.startswith(TARGET)])
            if len(sense_nums) == 1:    # 한 문장에 원하는 단어의 의미가 한가지로만 쓰인 문장만을 골라서
                try:
                    sense_num = int(list(sense_nums)[0])
                    if sense_num < 50:    # 의미 번호가 너무 크면 뭔가 에러가 있으므로 버리고
                        # 단어에 붙어있는 모든 의미 번호를 제거하고
                        no_sense_tag = [re.sub(r"""__x?\d\d/""", '/', morph) for morph in sent]
                        # 문장 맨 앞에 원하는 단어의 의미 번호를 함께 출력
                        print >> fout, '%d\t%s' % (sense_num, ' '.join(no_sense_tag))
                except ValueError:
                    # 간혹 의미 번호가 '연기__x01'과 같이 특수하게 부여된 것이 있는데, 이런건 버린다.
                    pass
            sent = []

!head -n5 sense_corpus.tag

9	일본/NNP 의/JKG 경우/NNG 현재/MAG 주택/NNG 내부/NNG 에/JKB 설치/NNG 하/XSV 는/ETM 열/NNG 감지/NNG 형/XSN 화재/NNG 감지기/NNG 는/JX 18/SN 년/NNB ,/SP 아파트/NNG 의/JKG 복도/NNG -/SS 계단/NNG 등/NNB 에/JKB 설치/NNG 하/XSV 는/ETM 연기/NNG 감지/NNG 형/XSN 화재/NNG 감지기/NNG 10/SN 년/NNB 으로/JKB 사용/NNG 기간/NNG 을/JKO 못/NNG 박/VV 고/EC 있/VX 다/EF ./SF
9	또/MAG 최근/NNG 캐나다/NNP 의/JKG 소비자/NNG 잡지/NNG '/SS 캐네디언/NNG 컨슈머/NNG '/SS 는/JX 연기/NNG 탐지기/NNG 의/JKG 경우/NNG 수명/NNG 이/JKS 5/SN ∼/SO 10/SN 년/NNB 이/VCP 고/EC ,/SP 전체/NNG 의/JKG 20/SN ∼/SO 30/SN %/SW 는/JX 10/SN 년/NNB 내/NNB 기능/NNG 이/JKS 멈추/VV ᆫ다고/EC 지적/NNG ,/SP 눈길/NNG 을/JKO 끌/VV ᆫ다/EF ./SF
9	연기/NNG 감지/NNG 형/XSN 이/VCP ᆫ/ETM 경우/NNG 제사/NNG 때/NNG 쓰/VV 는/ETM 향/NNG 3/SN 개/NNB 정도/NNG 를/JKO 한꺼번에/MAG 피우/VV 어/EC 경보기/NNG 에/JKB 갖/VV 다/EC 대/VX 면/EC 소리/NNG 가/JKS 나/VV ᆫ다/EF ./SF
5	서/NNP 청장/NNG 은/JX 특히/MAG 법인세/NNG 세무/NNG 조사/NNG 대상/NNG 기업/NNG 을/JKO 현재/NNG 의/JKG 두/MM 배/NNG 로/JKB 늘리/VV 고/EC 현재/MAG 60/SN 일/NNB 이/VCP 던/ETM 각/MM 기업/NNG 의/JKG 조사/NNG 기한/NNG 을/JKO 각/MM 지방/NNG 청장/NNG 재량/NNG 에/JKB 따르/VV 아/EC 필요/NNG 하/XSA 다고/EC 판단/NNG 하

8. 단어 의미 중의성 해결 - Part 1 - 전통적인 방법
----

In [ ]:
### 의미 코퍼스를 읽어들입니다.

sense_corpus = []
for line in open('sense_corpus.tag'):
    sense, features = line.split('\t', 1)
    sense_corpus.append((int(sense), features.split()))

In [ ]:
### 전부 바이너리 벡터로 표현합니다.

import numpy as npy

def to_bin_vector(vocabulary, features, trg):
    """
    vocabulary 사전을 이용하여 모든 자질을 하나의 bag-of-words 벡터로 표현하는 함수
    @param  vocabulary  vocabulary 사전
    @param  features    자질 목록
    @param  trg         중의성을 해소할 단어
    @return             벡터
    """
    vec_sum = npy.zeros([len(vocabulary),], dtype=npy.int8)    # vocabulary 크기 만큼 벡터를 0으로 초기화 합니다.
    for feature in features:
        if feature.startswith('%s/' % trg):
            continue
        vec_sum[vocabulary[feature]] = 1    # 자질 번호에 해당하는 벡터 값을 1로 설정해 줍니다.
    return vec_sum

all_features = set(sum([features for _, features in sense_corpus], []))    # 모든 자질을 합칩니다.
vocabulary = {feature: num for num, feature in enumerate(all_features)}    # 각 자질에 고유 번호를 부여하여 사전을 구축합니다.
print 'Number of vocabulary:', len(vocabulary)

# 자질을 전부 벡터로 표현합니다.
bin_train = [(sense, to_bin_vector(vocabulary, features, '연기')) for sense, features in sense_corpus]
print 'First training instance:'
print bin_train[0]

Number of vocabulary: 7628
First training instance:
(9, array([0, 0, 0, ..., 0, 0, 0], dtype=int8))


In [ ]:
### SVM을 이용하여 학습 및 평가를 수행하는 함수를 작성합니다.

from sklearn import cross_validation
from sklearn import svm

def evaluate(X, Y):
    """
    SVM을 이용하여 평가를 수행하는 함수
    @param  X  자질 벡터
    @param  Y  outcome
    @return    정확도
    """
    # 학습 코퍼스를 8:2의 비율로 학습/평가 코퍼스로 나눕니다.
    X_train, X_eval, y_train, y_eval = cross_validation.train_test_split(X, Y, test_size=0.2, random_state=0)
    svm_model = svm.LinearSVC()
    svm_model.fit(X_train, y_train)    # 학습 집합을 이용해 학습합니다.
    return svm_model.score(X_eval, y_eval)    # 평가 집합으로 정확도를 평가합니다.

In [ ]:
### 평가를 해봅니다.

evaluate([features for _, features in bin_train], [sense for sense, _ in bin_train])

# 오!

0.86868686868686873

![](http://icongal.com/gallery/image/38973/thumbs_up_thumbs_up_vote_like.png)

9. 단어 의미 중의성 해결 - Part 2 - word2vec 활용
----

In [ ]:
### 전부 word2vec을 이용하여 벡터로 표현합니다.

import gensim
import numpy as npy

def to_w2v_vector(model, features, trg):
    """
    word2vec 모델을 이용하여 모든 자질을 더해서 하나의 벡터로 표현하는 함수
    @param  model     word2vec 모델
    @param  features  자질 목록
    @param  trg       중의성을 해소할 단어
    @return           벡터
    """
    vec_sum = npy.zeros([model.vector_size,])    # word2vec 모델의 벡터 크기 만큼 벡터를 0으로 초기화 합니다.
    for feature in set(features):
        if feature.startswith('%s/' % trg):
            continue
        try:
            vec_sum += model[unicode(feature, 'UTF-8')]
        except KeyError:
            pass    # Out-Of-Vocabulary 자질들은 그냥 넘어갑니다.
    return vec_sum

# 원시 코퍼스로 pre-training 해둔 word2vec 모델을 읽어들입니다.
w2v_model = gensim.models.Word2Vec.load_word2vec_format('raw_corpus.w2v', binary=True)
print 'Vector size:', w2v_model.vector_size

# 자질을 전부 벡터로 표현합니다.
w2v_train = [(sense, to_w2v_vector(w2v_model, features, '연기')) for sense, features in sense_corpus]
print 'First training instance:'
print w2v_train[0]

Vector size: 200
First training instance:
(9, array([ 2.03780541, -0.05760974,  0.45845033,  0.16865686,  0.38574068,
        0.63337594, -1.1427569 , -0.10811525, -0.01907412, -1.21633177,
        0.47648489, -0.04538528,  0.17248241, -0.67216815, -0.58530399,
       -0.0795107 ,  0.11482168,  0.30661339,  0.45757701, -0.53109309,
       -0.20708619,  0.29574604, -0.84984107,  0.91007282, -0.54807297,
        0.86658714, -1.33023983,  0.15370091, -0.76278742, -0.29612444,
       -0.05138013,  0.64856726,  0.97502029,  0.17566016,  0.96827262,
        0.25050572, -1.01533307, -0.93904025, -0.57080799, -0.16608133,
       -0.58569992, -0.29802095,  0.04178477,  1.51128502, -0.10311706,
       -0.77958182, -0.19423184,  0.42869712, -0.04144236, -0.1677563 ,
        0.81574613, -1.20705579,  0.38499597, -0.38002745,  0.42406251,
        0.97285284,  0.49995938,  0.42171961, -1.15597911,  0.01568658,
       -0.61849618,  1.00722201,  1.04033946, -0.4688745 , -0.21817498,
        1.28075883

In [ ]:
### 평가를 해봅니다.

evaluate([features for _, features in w2v_train], [sense for sense, _ in w2v_train])

# 오오!

0.91245791245791241

![](http://i1.daumcdn.net/thumb/R750x0/?fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F2406FE4054BC833F2FA8F7)